How I plan to start this off. (Jean)

In [11]:
# Just took whatever libraries he put on from class notes so I don't go back and forth. Will change later (maybe?)
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV

In [12]:
#Load Data
train_df = pd.read_csv("train.csv.zip")
train_df.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [14]:
train_df['insult'].value_counts()

0    151694
1      7877
Name: insult, dtype: int64

In [37]:
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(train_df["comment_text"])
X_train_counts.shape


(159571, 189775)

In [16]:
count_vect.vocabulary_.get(u'algorithm')
#vectorizer.get_feature_names_out()

14404

In [19]:
count_vect.get_feature_names_out()
#count_vect.vocabulary_.get(u'algorithm')

14404

In [26]:
vectorizer2 = CountVectorizer(analyzer='word', ngram_range=(2, 2))
X2 = vectorizer2.fit_transform(train_df["comment_text"])
vectorizer2.get_feature_names_out()


array(['00 00', '00 000', '00 00pm', ..., '𐌰𐌹 he', '𐌰𐌿 on', '𐌴𐌹 to'],
      dtype=object)

In [25]:
X2.shape

(159571, 2277365)

In [38]:
vectorizer3 = CountVectorizer(lowercase=False)
X3 = vectorizer2.fit_transform(train_df["comment_text"])
X3.shape

(159571, 234532)

189775
234532

In [3]:
# Use the Count Vectorizer to grab the info and Features I would need. 
vec_cv = CountVectorizer(max_features=150)
tmp = vec_cv.fit_transform(train_df["comment_text"])
tok_cols = vec_cv.get_feature_names_out()
tok_df = pd.DataFrame(tmp.toarray(), columns=tok_cols)
print("original:", train_df["comment_text"].shape)
print("vectorized:", tmp.shape)
tok_df.sample(10)

original: (159571,)
vectorized: (159571, 150)


,about,add,after,again,all,also,am,an,and,any,...,who,why,wikipedia,will,with,work,would,wp,you,your
19206,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,1,0,0,0,0,0
129833,0,0,0,1,2,0,0,0,6,0,...,3,1,2,1,2,0,1,0,16,4
14420,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
154638,0,0,0,0,0,0,0,1,3,0,...,0,0,0,0,2,0,0,0,5,3
109078,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,1
150755,0,0,0,0,0,0,0,1,1,0,...,0,0,0,0,0,0,0,0,2,0
12169,4,0,0,0,2,2,0,1,17,2,...,0,0,1,0,1,0,1,0,12,6
79351,0,0,0,0,0,1,0,1,0,0,...,0,0,0,0,0,0,0,0,4,2
101310,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
83301,0,0,0,0,0,0,0,1,1,0,...,0,0,1,1,1,0,0,1,4,1


Counting the vectors:
- max 150: (159571, 150)
- no max: (159571, 189775)
### Wonder if there is a way to figure out the optimal amount. Or should we just leave it as is?

### Or more like stop even trying with count_vectorizing cause it's easier to use TF-IDF

In [5]:
#TF-IDF
vec_tf = TfidfVectorizer(max_features=150, strip_accents="unicode")
tmp = vec_tf.fit_transform(train_df["comment_text"])
tok_cols = vec_tf.get_feature_names_out()
tok_df = pd.DataFrame(tmp.toarray(), columns=tok_cols)
print("original:", train_df["comment_text"].shape)
print("vectorized:", tmp.shape)
tok_df.sample(10)

original: (159571,)
vectorized: (159571, 150)


,about,add,after,again,all,also,am,an,and,any,...,who,why,wikipedia,will,with,work,would,wp,you,your
46246,0.000000,0.0,0.000000,0.00000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.389361,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
157281,0.000000,0.0,0.000000,0.00000,0.0,0.000000,0.000000,0.200095,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.182259,0.000000,0.0,0.000000,0.130554,0.000000
78314,0.000000,0.0,0.000000,0.00000,0.0,0.145826,0.000000,0.000000,0.224753,0.000000,...,0.073511,0.076591,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
131782,0.000000,0.0,0.049342,0.00000,0.0,0.039278,0.000000,0.096196,0.040358,0.112746,...,0.000000,0.082518,0.133271,0.035764,0.058414,0.000000,0.0,0.000000,0.188293,0.030382
128267,0.000000,0.0,0.000000,0.00000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.218155,0.178161,0.000000,0.0,0.000000,0.382856,0.000000
159059,0.000000,0.0,0.000000,0.00000,0.0,0.000000,0.000000,0.000000,0.098591,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
34156,0.000000,0.0,0.000000,0.00000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.277603,0.000000
133628,0.263787,0.0,0.000000,0.00000,0.0,0.000000,0.000000,0.000000,0.235599,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.190981,0.000000,0.000000
138072,0.126911,0.0,0.000000,0.17604,0.0,0.000000,0.160001,0.000000,0.151133,0.140739,...,0.000000,0.000000,0.000000,0.000000,0.109376,0.184651,0.0,0.000000,0.078347,0.000000
109279,0.000000,0.0,0.000000,0.00000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000


In [8]:
vec_tf = TfidfVectorizer(max_features=1000, strip_accents="unicode")
tmp = vec_tf.fit_transform(train_df["comment_text"])
tok_cols = vec_tf.get_feature_names_out()
tok_df = pd.DataFrame(tmp.toarray(), columns=tok_cols)
print("original:", train_df["comment_text"].shape)
print("vectorized:", tmp.shape)
tok_df.sample(10)

original: (159571,)
vectorized: (159571, 1000)


,000,10,100,11,12,13,14,15,16,17,...,wrote,www,yeah,year,years,yes,yet,you,your,yourself
17140,0.000000,0.0,0.0,0.0,0.378563,0.0,0.38776,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000
148747,0.000000,0.0,0.0,0.0,0.000000,0.0,0.00000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.273311,0.0,0.199960,0.000000,0.000000
136442,0.000000,0.0,0.0,0.0,0.000000,0.0,0.00000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.432971,0.0,0.000000,0.000000,0.000000
125889,0.000000,0.0,0.0,0.0,0.000000,0.0,0.00000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000
118092,0.000000,0.0,0.0,0.0,0.000000,0.0,0.00000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000
83644,0.000000,0.0,0.0,0.0,0.000000,0.0,0.00000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.088412,0.000000,0.000000
72132,0.402588,0.0,0.0,0.0,0.000000,0.0,0.00000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.119230,0.000000,0.000000
124596,0.000000,0.0,0.0,0.0,0.000000,0.0,0.00000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000
136832,0.000000,0.0,0.0,0.0,0.000000,0.0,0.00000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.137283,0.000000
84937,0.000000,0.0,0.0,0.0,0.000000,0.0,0.00000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.198705,0.000000,0.052208


In [10]:
tok_df.columns

Index(['000', '10', '100', '11', '12', '13', '14', '15', '16', '17',
       ...
       'wrote', 'www', 'yeah', 'year', 'years', 'yes', 'yet', 'you', 'your',
       'yourself'],
      dtype='object', length=1000)

In [7]:
vec_tf = TfidfVectorizer(strip_accents="unicode")
tmp = vec_tf.fit_transform(train_df["comment_text"])
tok_cols = vec_tf.get_feature_names_out()
#tok_df = pd.DataFrame(tmp.toarray(), columns=tok_cols)
#print("original:", train_df["comment_text"].shape)
print("vectorized:", tmp.shape)
#tok_df.sample(10)

vectorized: (159571, 188662)


I'm comparing count vectorization and TF-IDF. Most likely going to use some sort of vectorization parameters. Most likely would be making a tokenizaer to figure out whether it is a threat or not.

In [6]:
import nltk

for package in ['stopwords','punkt','wordnet']:
    nltk.download(package)
    
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize

stop_words = set(stopwords.words('english')) 
stop_words

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Jean\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Jean\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Jean\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


{'a',
 'about',
 'above',
 'after',
 'again',
 'against',
 'ain',
 'all',
 'am',
 'an',
 'and',
 'any',
 'are',
 'aren',
 "aren't",
 'as',
 'at',
 'be',
 'because',
 'been',
 'before',
 'being',
 'below',
 'between',
 'both',
 'but',
 'by',
 'can',
 'couldn',
 "couldn't",
 'd',
 'did',
 'didn',
 "didn't",
 'do',
 'does',
 'doesn',
 "doesn't",
 'doing',
 'don',
 "don't",
 'down',
 'during',
 'each',
 'few',
 'for',
 'from',
 'further',
 'had',
 'hadn',
 "hadn't",
 'has',
 'hasn',
 "hasn't",
 'have',
 'haven',
 "haven't",
 'having',
 'he',
 'her',
 'here',
 'hers',
 'herself',
 'him',
 'himself',
 'his',
 'how',
 'i',
 'if',
 'in',
 'into',
 'is',
 'isn',
 "isn't",
 'it',
 "it's",
 'its',
 'itself',
 'just',
 'll',
 'm',
 'ma',
 'me',
 'mightn',
 "mightn't",
 'more',
 'most',
 'mustn',
 "mustn't",
 'my',
 'myself',
 'needn',
 "needn't",
 'no',
 'nor',
 'not',
 'now',
 'o',
 'of',
 'off',
 'on',
 'once',
 'only',
 'or',
 'other',
 'our',
 'ours',
 'ourselves',
 'out',
 'over',
 'own',
 'r

In [7]:
class swTokenizer(object):
    def __init__(self, stop_words):
        self.stop_words = stop_words
    def __call__(self, doc):
        tokens = word_tokenize(doc)
        filtered_tok = []
        for tok in tokens:
            if tok not in stop_words:
                filtered_tok.append(tok)
        return filtered_tok

In [8]:
class stemTokenizer(object):
    def __init__(self, stop_words):
        self.stop_words = stop_words
        from nltk.stem import SnowballStemmer
        self.stemmer = SnowballStemmer(language='english')
    def __call__(self, doc):
        tokens = word_tokenize(doc)
        filtered_tok = []
        for tok in tokens:
            if tok not in stop_words:
                filtered_tok.append(self.stemmer.stem(tok))
        return filtered_tok

In [9]:
class lemmaTokenizer(object):
    def __init__(self, stop_words):
        self.stop_words = stop_words
        from nltk.stem import WordNetLemmatizer
        self.lemmatizer = WordNetLemmatizer()
    def __call__(self, doc):
        tokens = word_tokenize(doc)
        filtered_tok = []
        for tok in tokens:
            if tok not in stop_words:
                filtered_tok.append(self.lemmatizer.lemmatize(tok))
        return filtered_tok

In [10]:
from sklearn.svm import SVC
vec_tf = TfidfVectorizer()
model_svc = SVC()

y = train_df["insult"].head(n=7000)
X = train_df["comment_text"].head(n=7000)

X_train, X_test, y_train, y_test = train_test_split(X, y)

pipe2 = Pipeline([ 
                    #("vect", vec_cv),
                    ("vect", vec_tf),
                    ("model", model_svc)
])

params = {"vect__max_features":[100,500,1000,1500],
            "vect__tokenizer":(swTokenizer(stop_words), stemTokenizer(stop_words), lemmaTokenizer(stop_words) ),
            "vect__norm":["l1","l2"]
            }

grid = GridSearchCV(estimator  = pipe2, 
                               param_grid = params, 
                               scoring    = "balanced_accuracy")

grid.fit(X_train, y_train.ravel())
best = grid.best_estimator_
preds = best.predict(X_test)
print(best)
print(classification_report(y_test, preds))
sns.heatmap(confusion_matrix(y_test, preds), annot=True)

c:\Users\Jean\anaconda3\Lib\site-packages\sklearn\feature_extraction\text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
c:\Users\Jean\anaconda3\Lib\site-packages\sklearn\feature_extraction\text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
c:\Users\Jean\anaconda3\Lib\site-packages\sklearn\feature_extraction\text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
c:\Users\Jean\anaconda3\Lib\site-packages\sklearn\feature_extraction\text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
c:\Users\Jean\anaconda3\Lib\site-packages\sklearn\feature_extraction\text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
c:\Users\Jean\anaconda3\Lib\site-packages\sklearn\

KeyboardInterrupt: 